In [0]:
ecom_oct_df = spark.read.csv('/Volumes/idc/idc_kaggle/ecom_data/2019-Oct.csv',header = True,inferSchema = True)

In [0]:
ecom_oct_df.printSchema()   
ecom_oct_df.count()

In [0]:
# Display basic info
print(f"October 2019 - Total Events: {ecom_oct_df.count():,}")
print("\n" + "="*60)
print("SCHEMA:")
print("="*60)
ecom_oct_df.printSchema()

print("\n" + "="*60)
print("SAMPLE DATA (First 5 rows):")
print("="*60)
ecom_oct_df.limit(5).display()

# Quick stats
print("\n" + "="*60)
print("EVENT TYPE DISTRIBUTION:")
print("="*60)
ecom_oct_df.groupBy("event_type").count().orderBy("count", ascending=False).display()

In [0]:
ecom_nov_df = spark.read.csv("/Volumes/idc/idc_kaggle/ecom_data/2019-Nov.csv",header = True,inferSchema = True)
ecom_nov_df.printSchema()
ecom_nov_df.count()

In [0]:
ecom_df = ecom_oct_df.union(ecom_nov_df)
print(f"Total Events: {ecom_df.count():,}")

In [0]:
from pyspark.sql import functions as F

print("="*60)
print("DATA QUALITY CHECK")
print("="*60)

null_count = ecom_df.select(
    [
        F.count(
            F.when(
                (
                    (F.col(c).isNull()) |
                    (F.isnan(F.col(c)) if dict(ecom_df.dtypes)[c] in ["float", "double"] else F.lit(False))
                ),
                c
            )
        ).alias(c)
        for c in ecom_df.columns
    ]
)
display(null_count)

In [0]:
print("\nNull counts by column:")
null_count.show(vertical=True)

In [0]:
ecom_df.select("category_code").where(F.col("category_code").isNotNull()).count()


In [0]:
ecom_df.select("category_code").where(F.col("category_code").isNull()).count()

In [0]:
# Check unique counts
print(f"\\nUnique Users: {ecom_df.select('user_id').distinct().count():,}")
print(f"Unique Products: {ecom_df.select('product_id').distinct().count():,}")
print(f"Unique Sessions: {ecom_df.select('user_session').distinct().count():,}")
print(f"Unique Brands: {ecom_df.select('brand').distinct().count():,}")


In [0]:
# Price statistics
print("\\nPrice Statistics:")
ecom_df.select("price").summary().show()

In [0]:
display(ecom_oct_df.limit(5))

In [0]:
display(ecom_nov_df.limit(5))